<a href="https://colab.research.google.com/github/Loggo-MediCare/python-numpy-stock/blob/main/mu_%E5%A4%9A%E7%AD%96%E7%95%A5%E6%95%B4%E5%90%88_%E7%B7%9A%E6%80%A7%E5%9B%9E%E6%AD%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import warnings
import yfinance as yf
import pandas_datareader.data as web

# 模型相關庫
from statsmodels.tsa.arima.model import ARIMA
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error

warnings.filterwarnings('ignore')

# =============================================================================
# 1. fix by gemini 技術指標計算函數 (保持不變，但在邏輯中增強使用)
# =============================================================================
def calculate_MA(prices, period=10):
    prices = np.array(prices, dtype=float)
    ma = np.full(len(prices), np.nan)
    for i in range(period - 1, len(prices)):
        ma[i] = np.mean(prices[i - period + 1:i + 1])
    return ma

def calculate_EMA(prices, period=12):
    prices = np.array(prices, dtype=float)
    ema = np.full(len(prices), np.nan)
    multiplier = 2 / (period + 1)
    ema[period - 1] = np.mean(prices[:period])
    for i in range(period, len(prices)):
        ema[i] = (prices[i] - ema[i - 1]) * multiplier + ema[i - 1]
    return ema

def calculate_RSI(prices, period=14):
    prices = np.array(prices, dtype=float)
    rsi = np.full(len(prices), np.nan)
    deltas = np.diff(prices)
    gains = np.where(deltas > 0, deltas, 0)
    losses = np.where(deltas < 0, -deltas, 0)

    # 初始計算
    avg_gain = np.mean(gains[:period])
    avg_loss = np.mean(losses[:period])

    if avg_loss == 0:
        rsi[period] = 100
    else:
        rs = avg_gain / avg_loss
        rsi[period] = 100 - (100 / (1 + rs))

    # 平滑計算
    for i in range(period + 1, len(prices)):
        gain = gains[i-1]
        loss = losses[i-1]
        avg_gain = (avg_gain * (period - 1) + gain) / period
        avg_loss = (avg_loss * (period - 1) + loss) / period

        if avg_loss == 0:
            rsi[i] = 100
        else:
            rs = avg_gain / avg_loss
            rsi[i] = 100 - (100 / (1 + rs))
    return rsi

def calculate_MACD(prices, fast_period=12, slow_period=26, signal_period=9):
    prices = np.array(prices, dtype=float)
    ema_fast = calculate_EMA(prices, fast_period)
    ema_slow = calculate_EMA(prices, slow_period)
    macd = ema_fast - ema_slow

    signal = np.full(len(prices), np.nan)
    # 計算 Signal 線
    # 需要先剔除前面的 NaN
    valid_start = slow_period - 1
    if len(macd) > valid_start + signal_period:
        valid_macd = macd[valid_start:]
        signal_values = calculate_EMA(valid_macd, signal_period)
        signal[valid_start:] = signal_values

    return macd, signal, macd - signal

# =============================================================================
# 2. 整合策略類 (核心優化部分)
# =============================================================================
class IntegratedTradingStrategy:
    def __init__(self, target_stock='mu', data_period_years=5):
        self.target_stock = target_stock.upper()
        # 相關性資產
        self.correlated_stocks = [self.target_stock, 'NVDA', 'AMD'] # 確保包含目標股票自身
        self.currency_pairs = ['DEXJPUS', 'DEXUSUK']
        self.indices = ['SP500', 'DJIA', 'VIXCLS']

        self.return_period = 5 # 預測未來 5 天 (一週)
        self.data_period_years = data_period_years

        # 參數
        self.ma_period = 20
        self.stop_loss_pct = 0.03
        self.take_profit_pct = 0.06

        # 模型儲存
        self.arimax_model = None
        self.linear_model = None
        self.best_model_name = None

    def load_data(self, start_date, end_date):
        print(f"[{self.target_stock}] 正在加載多因子數據 ({start_date} ~ {end_date})...")

        # 1. 下載股票數據 (強制 auto_adjust=False 以獲取 Adj Close)
        stk_data = yf.download(self.correlated_stocks, start=start_date, end=end_date, auto_adjust=False, progress=False)

        # 2. 處理 yfinance 複雜的 MultiIndex
        # 如果只有一支股票，columns 可能只是 Index，如果是多支，則是 MultiIndex
        if isinstance(stk_data.columns, pd.MultiIndex):
            # 嘗試標準化：保留 Adj Close，若無則用 Close
            try:
                price_data = stk_data['Adj Close'].copy()
            except KeyError:
                print("警告: 找不到 'Adj Close'，嘗試使用 'Close'")
                price_data = stk_data['Close'].copy()
        else:
            # 單支股票情況
            price_data = pd.DataFrame(stk_data['Adj Close'])
            price_data.columns = [self.target_stock]

        # 填補缺失值
        price_data = price_data.ffill().bfill()

        # 3. 下載總經數據 (FRED)
        try:
            ccy_data = web.DataReader(self.currency_pairs, 'fred', start_date, end_date).ffill()
            idx_data = web.DataReader(self.indices, 'fred', start_date, end_date).ffill()
        except Exception as e:
            print(f"FRED 數據下載失敗: {e} (將使用隨機數據模擬以免程式崩潰)")
            # 模擬數據以免中斷 (僅供調試用)
            dates = price_data.index
            ccy_data = pd.DataFrame(np.random.randn(len(dates), len(self.currency_pairs)), index=dates, columns=self.currency_pairs)
            idx_data = pd.DataFrame(np.random.randn(len(dates), len(self.indices)), index=dates, columns=self.indices)

        return price_data, ccy_data, idx_data

    def prepare_features(self, stk_prices, ccy_data, idx_data):
        print("正在準備 ARIMAX/LR 特徵工程...")

        # 合併所有數據並確保索引對齊
        all_data = pd.concat([stk_prices, ccy_data, idx_data], axis=1).ffill().dropna()

        # 1. 建構目標變量 Y (未來 N 天的收益率)
        # shift(-N) 表示將未來的數據對齊到今天
        target_series = all_data[self.target_stock]
        Y = np.log(target_series).diff(self.return_period).shift(-self.return_period)
        Y.name = 'Target_Return'

        # 2. 建構特徵變量 X (過去的數據)
        # 相關股票的收益率
        X1 = np.log(all_data[self.correlated_stocks]).diff(self.return_period)
        # 貨幣與指數的變動率
        X2 = np.log(all_data[self.currency_pairs]).diff(self.return_period)
        X3 = np.log(all_data[self.indices]).diff(self.return_period)

        # 動量特徵 (Momentum): 過去 1週, 3週, 6週 的收益率
        periods = [self.return_period, self.return_period*3, self.return_period*6]
        X4_list = []
        for p in periods:
            col = np.log(target_series).diff(p)
            col.name = f'Momentum_{p}d'
            X4_list.append(col)
        X4 = pd.concat(X4_list, axis=1)

        # 合併所有 X
        X = pd.concat([X1, X2, X3, X4], axis=1)

        # 清理 NaN (因為 diff 和 shift 會產生 NaN)
        dataset = pd.concat([Y, X], axis=1).dropna()

        # 為了避免序列相關性過高，我們每隔 return_period 取樣一次 (週頻率)
        dataset = dataset.iloc[::self.return_period, :]

        return dataset.drop(columns=['Target_Return']), dataset['Target_Return'], all_data

    def train_models(self, X_train, Y_train, X_test, Y_test):
        print("-> 正在訓練 AI 模型 (LR, LASSO, ARIMAX)...")

        # --- 1. 線性模型群 ---
        models = {
            'LR': LinearRegression(),
            'LASSO': Lasso(alpha=0.0001, max_iter=10000), # alpha設小一點避免全被懲罰掉
            'EN': ElasticNet(alpha=0.0001, max_iter=10000)
        }

        best_mse = float('inf')

        for name, model in models.items():
            model.fit(X_train, Y_train)
            pred = model.predict(X_test)
            mse = mean_squared_error(Y_test, pred)
            # print(f"   - {name} MSE: {mse:.6f}")

            if mse < best_mse:
                best_mse = mse
                self.best_model_name = name
                self.linear_model = model

        print(f"-> 最佳線性模型: {self.best_model_name} (MSE: {best_mse:.6f})")

        # --- 2. ARIMAX 模型 ---
        # 選擇部分重要特徵給 ARIMAX (避免維度災難)
        # 簡單起見，我們選相關性最高的 3 個特徵
        corrs = X_train.corrwith(Y_train).abs().sort_values(ascending=False)
        top_features = corrs.head(3).index.tolist()
        self.arimax_exog_vars = top_features

        try:
            # order=(p,d,q) 可以通過 auto_arima 優化，這裡固定為 (1,0,1)
            model_arima = ARIMA(endog=Y_train, exog=X_train[top_features], order=(1,0,1))
            self.arimax_model = model_arima.fit()

            # 測試 ARIMAX
            arima_pred = self.arimax_model.forecast(steps=len(X_test), exog=X_test[top_features])
            arima_mse = mean_squared_error(Y_test, arima_pred)
            print(f"-> ARIMAX (1,0,1) MSE: {arima_mse:.6f}")

        except Exception as e:
            print(f"ARIMAX 訓練失敗: {e}")

    def analyze_technical_status(self, full_price_data):
        """分析當前的技術面狀態 (MA, RSI, MACD)"""
        prices = full_price_data[self.target_stock].values

        ma = calculate_MA(prices, self.ma_period)
        rsi = calculate_RSI(prices, 14)
        macd, signal, hist = calculate_MACD(prices)

        # 取得最新值
        curr_price = prices[-1]
        curr_ma = ma[-1]
        curr_rsi = rsi[-1]
        curr_hist = hist[-1]

        status = {
            'price': curr_price,
            'rsi': curr_rsi,
            'macd_hist': curr_hist,
            'ma_trend': 'BULL' if curr_price > curr_ma else 'BEAR'
        }

        # 綜合技術評分 (-1 到 1)
        score = 0
        if curr_price > curr_ma: score += 1
        else: score -= 1

        if curr_hist > 0: score += 1
        else: score -= 1

        if 40 < curr_rsi < 60: score += 0 # 中性
        elif curr_rsi >= 60: score += 1 # 強勢 (但也可能超買)
        elif curr_rsi <= 40: score -= 1 # 弱勢

        status['tech_score'] = score # 範圍約 -3 ~ 3

        return status

    def run_strategy(self):
        print("\n" + "="*60)
        print(f"執行每週策略分析: {self.target_stock}")
        print("="*60)

        # 1. 準備數據
        end_date = datetime.now()
        start_date = end_date - timedelta(days=365 * self.data_period_years)

        stk_prices, ccy_data, idx_data = self.load_data(start_date, end_date)
        X, Y, raw_data = self.prepare_features(stk_prices, ccy_data, idx_data)

        # 2. 訓練模型 (滾動視窗：使用過去所有數據訓練，預測下一週)
        # 分割 80% 訓練, 20% 驗證 (此處為了示範簡單分割)
        train_size = int(len(X) * 0.85)
        X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]
        Y_train, Y_test = Y.iloc[:train_size], Y.iloc[train_size:]

        self.train_models(X_train, Y_train, X_test, Y_test)

        # 3. 生成下週預測 (AI Signal)
        # 構建當前最新的特徵向量 (Raw Data 的最後一行)
        latest_data = raw_data.iloc[-1:]
        # 重建特徵 (需要手動重新計算最後一行的 Diff，因為 prepare_features 裡的 diff 會讓最後一行變 NaN 如果沒 shift)
        # 這裡為了簡化，我們直接拿 X 的最後一行做近似 (實際生產環境需精確重算)
        current_features = X.iloc[-1:].values.reshape(1, -1)
        current_features_df = X.iloc[-1:]

        pred_lr = self.linear_model.predict(current_features)[0]

        pred_arima = 0
        if self.arimax_model:
            # ARIMAX forecast 需要 exog
            exog_latest = current_features_df[self.arimax_exog_vars]
            pred_arima = self.arimax_model.forecast(steps=1, exog=exog_latest).iloc[0]

        avg_pred_return = (pred_lr + pred_arima) / 2
        print(f"\n[AI 預測結果]")
        print(f"線性模型預測: {pred_lr*100:.2f}%")
        print(f"ARIMAX 預測 : {pred_arima*100:.2f}%")
        print(f"-> 綜合預測下週漲跌: {avg_pred_return*100:.2f}%")

        # 4. 技術面分析 (Technical Signal)
        tech_status = self.analyze_technical_status(raw_data)
        print(f"\n[技術面狀態]")
        print(f"當前價格: {tech_status['price']:.2f}")
        print(f"RSI: {tech_status['rsi']:.2f}")
        print(f"MACD柱狀圖: {tech_status['macd_hist']:.4f}")
        print(f"技術評分: {tech_status['tech_score']} (正為多，負為空)")

        # 5. 最終決策 (AI Confidence Filter Logic)
        print("\n" + "-"*40)
        print("最終交易建議")
        print("-"*40)

        decision = "觀望 (Wait)"
        action = "無動作"

        # 邏輯 A: 技術面與 AI 共振 (最佳進場點)
        if tech_status['tech_score'] >= 1 and avg_pred_return > 0.005:
            decision = "強烈買入 (Strong Buy)"
            action = "進場做多 (趨勢確認)"

        # 邏輯 B: AI 強力看漲，技術面尚未跟上 (左側交易/潛伏)
        # 條件：AI預測 > 1.5% 且 RSI 沒有超賣 (不是接刀) 且 MACD 柱狀圖在收斂
        elif avg_pred_return > 0.015 and tech_status['rsi'] > 30:
            decision = "潛伏買入 (Accumulate)"
            action = "小部位建倉 (博反彈)"

        # 邏輯 C: 技術面轉弱，AI 也看空
        elif tech_status['tech_score'] <= -1 and avg_pred_return < -0.005:
            decision = "賣出/做空 (Sell)"
            action = "清倉或反手"

        print(f"決策: {decision}")
        print(f"行動: {action}")
        print("="*60)

# =============================================================================
# 執行區
# =============================================================================
if __name__ == "__main__":
    # 可以將 'mu' 換成 'NVDA' 或其他美股代碼
    bot = IntegratedTradingStrategy(target_stock='mu', data_period_years=2)
    bot.run_strategy()


執行每週策略分析: MU
[MU] 正在加載多因子數據 (2023-11-29 08:16:21.909487 ~ 2025-11-28 08:16:21.909487)...
正在準備 ARIMAX/LR 特徵工程...
-> 正在訓練 AI 模型 (LR, LASSO, ARIMAX)...
-> 最佳線性模型: LR (MSE: 0.009344)
-> ARIMAX (1,0,1) MSE: 0.009540

[AI 預測結果]
線性模型預測: 0.53%
ARIMAX 預測 : 2.82%
-> 綜合預測下週漲跌: 1.67%

[技術面狀態]
當前價格: 230.26
RSI: 54.50
MACD柱狀圖: -3.5782
技術評分: -2 (正為多，負為空)

----------------------------------------
最終交易建議
----------------------------------------
決策: 潛伏買入 (Accumulate)
行動: 小部位建倉 (博反彈)


In [ ]:
# -*- coding: UTF-8 -*-
"""
================================================================================
策略名稱：多因子股價預測策略 (Multi-Factor Stock Prediction Strategy)
目標：股價預測 (Stock Price Prediction)
================================================================================

本策略整合多種方法論進行股價預測：

1. 技術指標分析 (Technical Analysis)
   - MA (移動平均線) - 趨勢判斷
   - RSI (相對強弱指標) - 超買超賣
   - MACD - 動量確認
   - 布林通道 - 波動率分析

2. 計量經濟模型 (Econometric Models)
   - ARIMAX - 時間序列預測
   - Linear Regression (LR) - 線性回歸
   - LASSO - L1 正則化回歸
   - ElasticNet (EN) - 彈性網路

3. 多因子分析 (Multi-Factor Analysis)
   - 相關股票報酬率 (avgo, GOOGL)
   - 匯率因子 (JPY/USD, GBP/USD)
   - 指數因子 (S&P500, DJIA, VIX)
   - 動量因子 (歷史報酬率)

4. 風險管理
   - 動態停利 (趨勢反轉出場)
   - 固定停利 (百分比目標)
   - 固定停損

策略週期：週線 (Weekly)
================================================================================
"""

import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import warnings

warnings.filterwarnings('ignore')

# =============================================================================
# 套件導入與檢查
# =============================================================================

# yfinance - 股票數據下載
try:
    import yfinance as yf
    HAS_YFINANCE = True
except ImportError:
    HAS_YFINANCE = False
    print("警告：未安裝 yfinance，將使用模擬數據 (pip install yfinance)")

# pandas_datareader - FRED 數據下載
try:
    import pandas_datareader.data as web
    HAS_DATAREADER = True
except ImportError:
    HAS_DATAREADER = False
    print("警告：未安裝 pandas_datareader，將使用模擬數據 (pip install pandas-datareader)")

# statsmodels - ARIMAX 模型
try:
    from statsmodels.tsa.arima.model import ARIMA
    HAS_STATSMODELS = True
except ImportError:
    HAS_STATSMODELS = False
    print("警告：未安裝 statsmodels，ARIMAX 模型將不可用 (pip install statsmodels)")

# sklearn - 機器學習模型
try:
    from sklearn.linear_model import LinearRegression, Lasso, ElasticNet
    from sklearn.metrics import mean_squared_error
    from sklearn.model_selection import KFold, cross_val_score
    HAS_SKLEARN = True
except ImportError:
    HAS_SKLEARN = False
    print("警告：未安裝 scikit-learn，回歸模型將不可用 (pip install scikit-learn)")


# =============================================================================
# 技術指標計算函數
# =============================================================================

def calculate_MA(prices, period=10):
    """
    計算移動平均線 (Moving Average)

    Args:
        prices: 價格序列 (numpy array 或 list)
        period: 計算週期

    Returns:
        MA 值序列
    """
    prices = np.array(prices, dtype=float)
    ma = np.full(len(prices), np.nan)
    for i in range(period - 1, len(prices)):
        ma[i] = np.mean(prices[i - period + 1:i + 1])
    return ma


def calculate_EMA(prices, period=12):
    """
    計算指數移動平均線 (Exponential Moving Average)

    Args:
        prices: 價格序列
        period: 計算週期

    Returns:
        EMA 值序列
    """
    prices = np.array(prices, dtype=float)
    ema = np.full(len(prices), np.nan)
    multiplier = 2 / (period + 1)
    ema[period - 1] = np.mean(prices[:period])
    for i in range(period, len(prices)):
        ema[i] = (prices[i] - ema[i - 1]) * multiplier + ema[i - 1]
    return ema


def calculate_RSI(prices, period=14):
    """
    計算相對強弱指標 (RSI)

    Args:
        prices: 價格序列
        period: 計算週期 (預設14)

    Returns:
        RSI 值序列 (0-100)
    """
    prices = np.array(prices, dtype=float)
    rsi = np.full(len(prices), np.nan)
    deltas = np.diff(prices)
    gains = np.where(deltas > 0, deltas, 0)
    losses = np.where(deltas < 0, -deltas, 0)
    for i in range(period, len(prices)):
        avg_gain = np.mean(gains[i - period:i])
        avg_loss = np.mean(losses[i - period:i])
        if avg_loss == 0:
            rsi[i] = 100
        else:
            rs = avg_gain / avg_loss
            rsi[i] = 100 - (100 / (1 + rs))
    return rsi


def calculate_MACD(prices, fast_period=12, slow_period=26, signal_period=9):
    """
    計算 MACD 指標

    Args:
        prices: 價格序列
        fast_period: 快線週期 (預設12)
        slow_period: 慢線週期 (預設26)
        signal_period: 訊號線週期 (預設9)

    Returns:
        (macd, signal, histogram) 三個序列
    """
    prices = np.array(prices, dtype=float)
    ema_fast = calculate_EMA(prices, fast_period)
    ema_slow = calculate_EMA(prices, slow_period)
    macd = ema_fast - ema_slow
    signal = np.full(len(prices), np.nan)
    valid_macd = macd[~np.isnan(macd)]
    if len(valid_macd) >= signal_period:
        signal_values = calculate_EMA(valid_macd, signal_period)
        start_idx = len(prices) - len(valid_macd)
        signal[start_idx:] = signal_values
    histogram = macd - signal
    return macd, signal, histogram


def calculate_Bollinger_Bands(prices, period=20, num_std=2):
    """
    計算布林通道

    Args:
        prices: 價格序列
        period: 計算週期 (預設20)
        num_std: 標準差倍數 (預設2)

    Returns:
        (upper, middle, lower) 三個序列
    """
    prices = np.array(prices, dtype=float)
    middle = calculate_MA(prices, period)
    upper = np.full(len(prices), np.nan)
    lower = np.full(len(prices), np.nan)
    for i in range(period - 1, len(prices)):
        std = np.std(prices[i - period + 1:i + 1])
        upper[i] = middle[i] + num_std * std
        lower[i] = middle[i] - num_std * std
    return upper, middle, lower


def calculate_KD(high, low, close, k_period=9, d_period=3):
    """
    計算 KD 隨機指標

    Args:
        high: 最高價序列
        low: 最低價序列
        close: 收盤價序列
        k_period: K值計算週期
        d_period: D值平滑週期

    Returns:
        (K, D) 兩個序列
    """
    high = np.array(high, dtype=float)
    low = np.array(low, dtype=float)
    close = np.array(close, dtype=float)

    n = len(close)
    rsv = np.full(n, np.nan)
    k = np.full(n, np.nan)
    d = np.full(n, np.nan)

    for i in range(k_period - 1, n):
        highest_high = np.max(high[i - k_period + 1:i + 1])
        lowest_low = np.min(low[i - k_period + 1:i + 1])
        if highest_high != lowest_low:
            rsv[i] = (close[i] - lowest_low) / (highest_high - lowest_low) * 100
        else:
            rsv[i] = 50

    k[k_period - 1] = rsv[k_period - 1]
    for i in range(k_period, n):
        if not np.isnan(rsv[i]):
            k[i] = (2/3) * k[i-1] + (1/3) * rsv[i]

    d[k_period + d_period - 2] = k[k_period + d_period - 2]
    for i in range(k_period + d_period - 1, n):
        if not np.isnan(k[i]):
            d[i] = (2/3) * d[i-1] + (1/3) * k[i]

    return k, d


# =============================================================================
# 數據生成函數
# =============================================================================

def generate_sample_data(periods=100):
    """
    生成範例資料用於測試

    Args:
        periods: 資料筆數

    Returns:
        DataFrame 包含 OHLCV 資料
    """
    np.random.seed(42)

    base_price = 100
    returns = np.random.randn(periods) * 0.02
    prices = base_price * np.cumprod(1 + returns)

    data = {
        'Open': prices * (1 + np.random.randn(periods) * 0.005),
        'High': prices * (1 + np.abs(np.random.randn(periods)) * 0.01),
        'Low': prices * (1 - np.abs(np.random.randn(periods)) * 0.01),
        'Close': prices,
        'Volume': np.random.randint(100000, 1000000, periods)
    }

    dates = pd.date_range(end=datetime.now(), periods=periods, freq='W')
    df = pd.DataFrame(data, index=dates)

    return df


# =============================================================================
# ARIMAX 多因子策略類別
# =============================================================================

class ARIMAXStrategy:
    """
    多因子 ARIMAX/線性回歸 策略

    使用多種因子進行股價預測：
    - 相關股票報酬率 (avgo, GOOGL)
    - 匯率因子 (JPY/USD, GBP/USD)
    - 指數因子 (S&P500, DJIA, VIX)
    - 動量因子 (歷史報酬率)
    """

    def __init__(self, target_stock='avgo'):
        """
        初始化 ARIMAX 策略

        Args:
            target_stock: 目標股票代碼 (預設: mu)
        """
        self.target_stock = target_stock.upper()
        self.correlated_stocks = ['mu', 'GOOGL']
        self.currency_pairs = ['DEXJPUS', 'DEXUSUK']  # JPY/USD, GBP/USD
        self.indices = ['SP500', 'DJIA', 'VIXCLS']     # S&P500, 道瓊, VIX
        self.return_period = 5  # 5日報酬率

        # 模型存儲
        self.best_regressor = None
        self.best_model_name = None
        self.arimax_model = None
        self.arimax_exog_vars = None
        self.feature_data = None

    def load_data(self, start_date, end_date):
        """
        加載股票、貨幣和指數數據

        Args:
            start_date: 開始日期
            end_date: 結束日期

        Returns:
            (股票數據, 貨幣數據, 指數數據)
        """
        print("正在加載 ARIMAX 所需的多因子數據...")

        if HAS_YFINANCE and HAS_DATAREADER:
            try:
                # 下載股票數據
                stk_tickers = [self.target_stock] + self.correlated_stocks
                stk_tickers = list(set(stk_tickers))  # 去重
                stk_data = yf.download(stk_tickers, start=start_date, end=end_date, progress=False)

                # 下載 FRED 數據 (貨幣和指數)
                ccy_data = web.DataReader(self.currency_pairs, 'fred', start_date, end_date)
                idx_data = web.DataReader(self.indices, 'fred', start_date, end_date)

                print(f"  股票數據: {len(stk_data)} 筆")
                print(f"  貨幣數據: {len(ccy_data)} 筆")
                print(f"  指數數據: {len(idx_data)} 筆")

                return stk_data, ccy_data, idx_data

            except Exception as e:
                print(f"  數據下載失敗: {e}")
                print("  使用模擬數據...")

        # 生成模擬數據
        return self._generate_mock_data(start_date, end_date)

    def _generate_mock_data(self, start_date, end_date):
        """生成模擬數據用於測試"""
        dates = pd.date_range(start=start_date, end=end_date, freq='B')
        n = len(dates)

        np.random.seed(42)

        # 模擬股票數據
        stk_tickers = list(set([self.target_stock] + self.correlated_stocks))
        stk_prices = {}
        for ticker in stk_tickers:
            base_price = 100 + np.random.rand() * 400
            returns = np.random.randn(n) * 0.02
            prices = base_price * np.cumprod(1 + returns)
            stk_prices[ticker] = prices

        stk_data = pd.DataFrame(stk_prices, index=dates)
        stk_data.columns = pd.MultiIndex.from_product([['Adj Close'], stk_data.columns])

        # 模擬貨幣數據
        ccy_data = pd.DataFrame({
            'DEXJPUS': 110 + np.random.randn(n).cumsum() * 0.5,
            'DEXUSUK': 1.3 + np.random.randn(n).cumsum() * 0.01
        }, index=dates)

        # 模擬指數數據
        idx_data = pd.DataFrame({
            'SP500': 4000 + np.random.randn(n).cumsum() * 20,
            'DJIA': 33000 + np.random.randn(n).cumsum() * 100,
            'VIXCLS': 20 + np.abs(np.random.randn(n)) * 5
        }, index=dates)

        print("  使用模擬數據進行測試")

        return stk_data, ccy_data, idx_data

    def prepare_features(self, stk_data_multi_index, ccy_data, idx_data):
        """
        準備特徵變量和目標變量

        Args:
            stk_data_multi_index: 股票數據 (MultiIndex)
            ccy_data: 貨幣數據
            idx_data: 指數數據

        Returns:
            (X, Y, dataset) - 特徵、目標、完整數據集
        """
        print("正在準備 ARIMAX/LR 特徵...")

        # 提取股票價格
        if 'Adj Close' in stk_data_multi_index.columns.get_level_values(0):
            stk_price_data = stk_data_multi_index['Adj Close'].copy()
        elif 'Close' in stk_data_multi_index.columns.get_level_values(0):
            print("  警告: 'Adj Close' 不存在, 使用 'Close' 價格")
            stk_price_data = stk_data_multi_index['Close'].copy()
        else:
            raise ValueError("錯誤: 找不到 'Adj Close' 或 'Close' 價格數據")

        # 合併所有數據
        all_data = pd.concat([stk_price_data, ccy_data, idx_data], axis=1).ffill().dropna()

        # 目標變量: 未來 N 日的對數報酬率
        Y = np.log(all_data.loc[:, self.target_stock]).diff(self.return_period).shift(-self.return_period)
        Y.name = f'{self.target_stock}_pred'

        # 特徵 X1: 相關股票報酬率
        X1 = np.log(all_data.loc[:, self.correlated_stocks]).diff(self.return_period)

        # 特徵 X2: 貨幣報酬率
        X2 = np.log(all_data.loc[:, self.currency_pairs]).diff(self.return_period)

        # 特徵 X3: 指數報酬率
        X3 = np.log(all_data.loc[:, self.indices]).diff(self.return_period)

        # 特徵 X4: 動量因子 (歷史報酬率)
        X4 = pd.concat([
            np.log(all_data.loc[:, self.target_stock]).diff(i)
            for i in [self.return_period, self.return_period*3,
                     self.return_period*6, self.return_period*12]
        ], axis=1)
        X4.columns = [f'{self.target_stock}_DT', f'{self.target_stock}_3DT',
                      f'{self.target_stock}_6DT', f'{self.target_stock}_12DT']

        # 合併所有特徵
        X = pd.concat([X1, X2, X3, X4], axis=1)

        # 建立完整數據集並清理
        dataset = pd.concat([Y, X], axis=1).dropna().iloc[::self.return_period, :]
        Y_final = dataset.loc[:, Y.name]
        X_final = dataset.loc[:, X.columns]

        self.feature_data = dataset

        print(f"  特徵數量: {len(X_final.columns)}")
        print(f"  樣本數量: {len(X_final)}")

        return X_final, Y_final, dataset

    def compare_models(self, X_train, Y_train, X_test, Y_test):
        """
        訓練並比較 LR, LASSO, EN 模型

        Args:
            X_train, Y_train: 訓練數據
            X_test, Y_test: 測試數據

        Returns:
            (best_regressor, best_name, best_mse)
        """
        if not HAS_SKLEARN:
            print("  警告: sklearn 未安裝，跳過模型比較")
            return None, None, float('inf')

        print("-> 正在訓練 LR, LASSO, EN 模型...")

        models = [
            ('LR', LinearRegression()),
            ('LASSO', Lasso(random_state=42, max_iter=10000)),
            ('EN', ElasticNet(random_state=42, max_iter=10000))
        ]

        best_test_mse = float("inf")
        best_model_name = None
        best_regressor = None

        for name, model in models:
            try:
                model.fit(X_train, Y_train)
                test_pred = model.predict(X_test)
                test_mse = mean_squared_error(Y_test, test_pred)

                print(f'   - {name}: Test MSE = {test_mse:.6f}')

                if test_mse < best_test_mse:
                    best_test_mse = test_mse
                    best_model_name = name
                    best_regressor = model

            except Exception as e:
                print(f'   - {name}: 訓練失敗 ({e})')
                continue

        if best_regressor:
            print(f"-> 最佳簡單模型: {best_model_name} (Test MSE: {best_test_mse:.6f})")
            self._print_regression_equation(best_regressor, X_train.columns)

        self.best_regressor = best_regressor
        self.best_model_name = best_model_name

        return best_regressor, best_model_name, best_test_mse

    def _print_regression_equation(self, model, feature_names):
        """印出迴歸方程式"""
        if hasattr(model, 'coef_') and hasattr(model, 'intercept_'):
            coefficients = model.coef_
            intercept = model.intercept_

            print("\n-> 最佳模型的迴歸線 (Regression Line):")
            equation = f"   Y_pred = {intercept:.6f}"

            for name, coef in zip(feature_names, coefficients):
                if abs(coef) > 1e-6:
                    sign = "+" if coef > 0 else "-"
                    equation += f" {sign} {abs(coef):.6f} * {name}"

            print(equation)

    def train_arimax_model(self, X_train, Y_train, order=(2, 0, 1)):
        """
        訓練 ARIMAX 模型

        Args:
            X_train: 訓練特徵
            Y_train: 訓練目標
            order: ARIMA 階數 (p, d, q)

        Returns:
            (model_fit, exogenous_vars)
        """
        if not HAS_STATSMODELS:
            print("  警告: statsmodels 未安裝，跳過 ARIMAX 訓練")
            return None, []

        print(f"-> 正在訓練 ARIMAX{order} 模型...")

        # 外生變量：排除目標股票自身的動量因子
        exogenous_vars = [col for col in X_train.columns if not col.startswith(self.target_stock)]
        X_train_arima = X_train.loc[:, exogenous_vars]

        try:
            model = ARIMA(endog=Y_train, exog=X_train_arima, order=order)
            model_fit = model.fit()

            self.arimax_model = model_fit
            self.arimax_exog_vars = exogenous_vars

            print(f"   - ARIMAX 訓練成功")

            return model_fit, exogenous_vars

        except Exception as e:
            print(f"   - ARIMAX 模型訓練失敗: {e}")
            return None, exogenous_vars

    def run_simple_model_strategy(self, data_period_years=5):
        """
        執行 LR/ARIMAX 策略的完整流程

        Args:
            data_period_years: 數據年數

        Returns:
            (model_info, predictions)
        """
        TICKER = self.target_stock
        end_date = datetime.now().strftime('%Y-%m-%d')
        start_date = (datetime.now() - timedelta(days=365 * data_period_years)).strftime('%Y-%m-%d')

        print("\n" + "=" * 60)
        print(f"週三訓練：線性回歸與 ARIMAX ({TICKER})")
        print("=" * 60)

        # 1. 加載數據
        stk_data, ccy_data, idx_data = self.load_data(start_date, end_date)

        # 2. 準備特徵
        X, Y, dataset = self.prepare_features(stk_data, ccy_data, idx_data)

        # 3. 分割訓練/測試集
        train_size = int(len(X) * 0.8)
        X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]
        Y_train, Y_test = Y.iloc[:train_size], Y.iloc[train_size:]

        print(f"-> 訓練集大小: {len(X_train)} 週")
        print(f"-> 測試集大小: {len(X_test)} 週")

        # 4. 比較線性模型
        best_regressor, best_name, best_mse = self.compare_models(X_train, Y_train, X_test, Y_test)

        # 5. 訓練 ARIMAX
        arimax_model, arimax_exog_vars = self.train_arimax_model(X_train, Y_train, order=(2, 0, 1))

        arimax_test_mse = float('inf')
        if arimax_model and len(X_test) > 0:
            try:
                X_test_arima = X_test.loc[:, arimax_exog_vars]
                test_forecast = arimax_model.forecast(steps=len(X_test), exog=X_test_arima)
                arimax_test_mse = mean_squared_error(Y_test, test_forecast)
                print(f"-> ARIMAX (2,0,1) Test MSE: {arimax_test_mse:.6f}")
            except Exception as e:
                print(f"-> ARIMAX 預測失敗: {e}")

        # 6. 生成預測
        predictions = {}
        current_features = X.iloc[-1:].fillna(0)

        if best_regressor:
            lr_prediction = best_regressor.predict(current_features)[0]
            predictions['linear_model'] = {
                'name': best_name,
                'prediction': lr_prediction,
                'mse': best_mse
            }
            print(f"\n-> 線性模型 ({best_name}) 下週預測報酬率: {lr_prediction:.4f} ({lr_prediction*100:.2f}%)")

        if arimax_model and arimax_exog_vars:
            try:
                arimax_pred = arimax_model.forecast(
                    steps=1,
                    exog=current_features.loc[:, arimax_exog_vars]
                )[0]
                predictions['arimax'] = {
                    'prediction': arimax_pred,
                    'mse': arimax_test_mse
                }
                print(f"-> ARIMAX 下週預測報酬率: {arimax_pred:.4f} ({arimax_pred*100:.2f}%)")
            except Exception as e:
                print(f"-> ARIMAX 預測失敗: {e}")

        print("\n" + "=" * 60)
        print("週三訓練完成，模型已準備好進行過濾！")
        print("=" * 60)

        model_info = {
            'target_stock': TICKER,
            'best_linear_model': best_name,
            'linear_mse': best_mse,
            'arimax_mse': arimax_test_mse,
            'train_size': len(X_train),
            'test_size': len(X_test)
        }

        return model_info, predictions


# =============================================================================
# 技術指標策略類別 (含動態停利)
# =============================================================================

class StockPredictionStrategy:
    """
    MA + RSI + MACD 綜合週策略 (含動態停利)

    整合技術分析與 ARIMAX 多因子模型的股價預測策略

    策略邏輯：

    進場條件：
    1. RSI > 50 且 MACD > Signal (多頭趨勢)
       - 價格由下往上穿越 MA → 做多
    2. RSI < 50 且 MACD < Signal (空頭趨勢)
       - 價格由上往下穿越 MA → 做空

    出場條件 (可選擇動態或固定停利)：
    - 動態停利：價格跌破/突破 MA (趨勢反轉) - 抓大波段
    - 固定停利：達到設定百分比 - 抓小波段
    - 停損：進場價的設定百分比
    """

    def __init__(self,
                 ma_period=10,
                 rsi_period=14,
                 macd_fast=12,
                 macd_slow=26,
                 macd_signal=9,
                 stop_loss_pct=0.03,
                 take_profit_pct=0.06,
                 use_dynamic_tp=True):
        """
        初始化策略參數

        Args:
            ma_period: MA 計算週期
            rsi_period: RSI 計算週期
            macd_fast: MACD 快線週期
            macd_slow: MACD 慢線週期
            macd_signal: MACD 訊號線週期
            stop_loss_pct: 停損百分比 (預設 3%)
            take_profit_pct: 固定停利百分比 (預設 6%)
            use_dynamic_tp: 是否使用動態停利 (True: 趨勢停利, False: 固定停利)
        """
        self.ma_period = ma_period
        self.rsi_period = rsi_period
        self.macd_fast = macd_fast
        self.macd_slow = macd_slow
        self.macd_signal = macd_signal
        self.stop_loss_pct = stop_loss_pct
        self.take_profit_pct = take_profit_pct
        self.use_dynamic_tp = use_dynamic_tp

        # 交易狀態
        self.position = 0       # 0: 空倉, 1: 多單, -1: 空單
        self.entry_price = 0
        self.entry_time = None

        # 績效記錄
        self.trades = []
        self.total_profit = 0

        # ARIMAX 策略 (可選)
        self.arimax_strategy = None
        self.arimax_predictions = None

    def calculate_indicators(self, data):
        """
        計算所有技術指標

        Args:
            data: DataFrame 包含 OHLCV 資料

        Returns:
            包含所有指標的 DataFrame
        """
        df = data.copy()

        # 確定收盤價欄位名稱
        close_col = 'Close' if 'Close' in df.columns else 'close'
        close_prices = df[close_col].values

        # 計算技術指標
        df['MA'] = calculate_MA(close_prices, self.ma_period)
        df['RSI'] = calculate_RSI(close_prices, self.rsi_period)

        macd, signal, hist = calculate_MACD(
            close_prices,
            self.macd_fast,
            self.macd_slow,
            self.macd_signal
        )
        df['MACD'] = macd
        df['MACD_Signal'] = signal
        df['MACD_Hist'] = hist

        upper, middle, lower = calculate_Bollinger_Bands(close_prices, 20, 2)
        df['BB_Upper'] = upper
        df['BB_Middle'] = middle
        df['BB_Lower'] = lower

        return df

    def generate_signal(self, df, i):
        """
        生成交易訊號 (含動態停利邏輯)

        Args:
            df: 包含指標的 DataFrame
            i: 當前索引

        Returns:
            訊號: 1 (買入), -1 (賣出), 2 (空單出場), -2 (多單出場), 0 (無動作)
        """
        if i < 1:
            return 0

        close_col = 'Close' if 'Close' in df.columns else 'close'

        current_price = df[close_col].iloc[i]
        prev_price = df[close_col].iloc[i-1]
        current_ma = df['MA'].iloc[i]
        prev_ma = df['MA'].iloc[i-1]
        current_rsi = df['RSI'].iloc[i]
        current_macd = df['MACD'].iloc[i]
        current_macd_signal = df['MACD_Signal'].iloc[i]

        if np.isnan(current_ma) or np.isnan(current_rsi) or np.isnan(current_macd):
            return 0

        # === 進場邏輯 ===
        if self.position == 0:
            # 多頭條件：RSI > 50 且 MACD > Signal，價格突破 MA
            if current_rsi > 50 and current_macd > current_macd_signal:
                if prev_price <= prev_ma and current_price > current_ma:
                    return 1  # 買入訊號

            # 空頭條件：RSI < 50 且 MACD < Signal，價格跌破 MA
            elif current_rsi < 50 and current_macd < current_macd_signal:
                if prev_price >= prev_ma and current_price < current_ma:
                    return -1  # 賣出訊號

        # === 出場邏輯 ===

        # 1. 停損檢查 (固定停損)
        if self.position == 1:
            if current_price <= self.entry_price * (1 - self.stop_loss_pct):
                return -2  # 多單停損出場

        if self.position == -1:
            if current_price >= self.entry_price * (1 + self.stop_loss_pct):
                return 2   # 空單停損出場

        # 2. 停利檢查
        if self.use_dynamic_tp:
            # === 動態停利：趨勢反轉時出場 (抓大波段/嘎空) ===

            # 多單出場條件：價格跌破 MA (趨勢反轉)
            if self.position == 1:
                if prev_price >= prev_ma and current_price < current_ma:
                    return -2

            # 空單出場條件：價格突破 MA (趨勢反轉)
            if self.position == -1:
                if prev_price <= prev_ma and current_price > current_ma:
                    return 2
        else:
            # === 固定停利：達到目標百分比 (抓小波段) ===

            if self.position == 1:
                if current_price >= self.entry_price * (1 + self.take_profit_pct):
                    return -2  # 多單停利出場

            if self.position == -1:
                if current_price <= self.entry_price * (1 - self.take_profit_pct):
                    return 2   # 空單停利出場

        return 0

    def execute_trade(self, signal, price, time):
        """
        執行交易

        Args:
            signal: 交易訊號
            price: 當前價格
            time: 當前時間

        Returns:
            交易記錄 (dict) 或 None
        """
        trade_record = None

        if signal == 1:  # 買入
            self.position = 1
            self.entry_price = price
            self.entry_time = time
            trade_record = {
                'type': 'BUY',
                'time': time,
                'price': price,
                'action': 'ENTRY'
            }

        elif signal == -1:  # 賣出
            self.position = -1
            self.entry_price = price
            self.entry_time = time
            trade_record = {
                'type': 'SELL',
                'time': time,
                'price': price,
                'action': 'ENTRY'
            }

        elif signal == -2:  # 多單出場
            profit = price - self.entry_price
            self.total_profit += profit
            trade_record = {
                'type': 'SELL',
                'time': time,
                'price': price,
                'action': 'EXIT',
                'entry_price': self.entry_price,
                'profit': profit
            }
            self.trades.append(trade_record)
            self.position = 0
            self.entry_price = 0

        elif signal == 2:  # 空單出場
            profit = self.entry_price - price
            self.total_profit += profit
            trade_record = {
                'type': 'BUY',
                'time': time,
                'price': price,
                'action': 'EXIT',
                'entry_price': self.entry_price,
                'profit': profit
            }
            self.trades.append(trade_record)
            self.position = 0
            self.entry_price = 0

        return trade_record

    def backtest(self, data):
        """
        執行回測

        Args:
            data: DataFrame 包含 OHLCV 資料

        Returns:
            回測結果字典
        """
        # 重置狀態
        self.position = 0
        self.entry_price = 0
        self.trades = []
        self.total_profit = 0

        # 計算指標
        df = self.calculate_indicators(data)
        close_col = 'Close' if 'Close' in df.columns else 'close'

        # 逐筆執行回測
        for i in range(len(df)):
            signal = self.generate_signal(df, i)

            if signal != 0:
                time = df.index[i].strftime('%Y-%m-%d') if hasattr(df.index[i], 'strftime') else str(i)
                price = df[close_col].iloc[i]
                self.execute_trade(signal, price, time)

        # 強制平倉最後的部位
        if self.position != 0:
            last_price = df[close_col].iloc[-1]
            last_time = df.index[-1].strftime('%Y-%m-%d') if hasattr(df.index[-1], 'strftime') else str(len(df)-1)

            if self.position == 1:
                profit = last_price - self.entry_price
            else:
                profit = self.entry_price - last_price

            self.total_profit += profit
            self.trades.append({
                'type': 'FORCED_EXIT',
                'time': last_time,
                'price': last_price,
                'action': 'EXIT',
                'entry_price': self.entry_price,
                'profit': profit
            })

        return self.get_performance_report()

    def get_performance_report(self):
        """
        生成績效報告

        Returns:
            績效報告字典
        """
        if len(self.trades) == 0:
            return {
                'total_trades': 0,
                'total_profit': 0,
                'win_rate': 0,
                'avg_profit': 0,
                'max_profit': 0,
                'max_loss': 0,
                'profit_factor': 0
            }

        profits = [t.get('profit', 0) for t in self.trades]
        wins = [p for p in profits if p > 0]
        losses = [p for p in profits if p <= 0]
        total_loss = sum(losses)

        return {
            'total_trades': len(profits),
            'winning_trades': len(wins),
            'losing_trades': len(losses),
            'total_profit': self.total_profit,
            'win_rate': len(wins) / len(profits) * 100 if profits else 0,
            'avg_profit': np.mean(profits) if profits else 0,
            'avg_win': np.mean(wins) if wins else 0,
            'avg_loss': np.mean(losses) if losses else 0,
            'max_profit': max(profits) if profits else 0,
            'max_loss': min(profits) if profits else 0,
            'profit_factor': abs(sum(wins) / total_loss) if total_loss != 0 else float('inf')
        }

    def predict_trend(self, df):
        """
        股價預測 - 趨勢判斷 (技術指標)

        Args:
            df: 包含指標的 DataFrame

        Returns:
            預測結果字典
        """
        if len(df) < 2:
            return {'prediction': 'NEUTRAL', 'confidence': 0}

        last_idx = len(df) - 1
        close_col = 'Close' if 'Close' in df.columns else 'close'

        current_price = df[close_col].iloc[last_idx]
        current_ma = df['MA'].iloc[last_idx]
        current_rsi = df['RSI'].iloc[last_idx]
        current_macd = df['MACD'].iloc[last_idx]
        current_macd_signal = df['MACD_Signal'].iloc[last_idx]
        current_bb_upper = df['BB_Upper'].iloc[last_idx]
        current_bb_lower = df['BB_Lower'].iloc[last_idx]

        signals = []
        confidence_factors = []

        # MA 訊號
        if not np.isnan(current_ma):
            if current_price > current_ma:
                signals.append(1)
            else:
                signals.append(-1)
            confidence_factors.append(abs(current_price - current_ma) / current_ma)

        # RSI 訊號
        if not np.isnan(current_rsi):
            if current_rsi > 70:
                signals.append(-1)  # 超買
                confidence_factors.append((current_rsi - 50) / 50)
            elif current_rsi < 30:
                signals.append(1)   # 超賣
                confidence_factors.append((50 - current_rsi) / 50)
            elif current_rsi > 50:
                signals.append(1)
                confidence_factors.append((current_rsi - 50) / 50)
            else:
                signals.append(-1)
                confidence_factors.append((50 - current_rsi) / 50)

        # MACD 訊號
        if not np.isnan(current_macd) and not np.isnan(current_macd_signal):
            if current_macd > current_macd_signal:
                signals.append(1)
            else:
                signals.append(-1)
            confidence_factors.append(0.3)

        # 布林通道訊號
        if not np.isnan(current_bb_upper) and not np.isnan(current_bb_lower):
            bb_width = current_bb_upper - current_bb_lower
            bb_position = (current_price - current_bb_lower) / bb_width if bb_width > 0 else 0.5

            if bb_position > 0.8:
                signals.append(-1)
            elif bb_position < 0.2:
                signals.append(1)
            confidence_factors.append(abs(bb_position - 0.5))

        if len(signals) == 0:
            return {'prediction': 'NEUTRAL', 'confidence': 0}

        avg_signal = np.mean(signals)
        avg_confidence = np.mean(confidence_factors) * 100

        if avg_signal > 0.3:
            prediction = 'BULLISH'
        elif avg_signal < -0.3:
            prediction = 'BEARISH'
        else:
            prediction = 'NEUTRAL'

        return {
            'prediction': prediction,
            'confidence': min(avg_confidence, 100),
            'signal_strength': avg_signal,
            'indicators': {
                'MA': current_ma,
                'RSI': current_rsi,
                'MACD': current_macd,
                'MACD_Signal': current_macd_signal
            }
        }

    def run_weekly_strategy(self, data=None, target_stock='avgo', include_arimax=True):
        """
        執行每週策略分析 (整合技術指標 + ARIMAX)

        Args:
            data: DataFrame 包含 OHLCV 資料 (若為 None 則生成測試資料)
            target_stock: 目標股票代碼
            include_arimax: 是否包含 ARIMAX 分析

        Returns:
            (model_info, results)
        """
        print("=" * 70)
        print("執行每週策略分析 - 多因子股價預測")
        print("=" * 70)

        # 1. 載入/生成數據
        print("\n[1/6] 生成/載入市場資料...")
        if data is None:
            data = generate_sample_data(100)

        print(f"  資料筆數: {len(data)}")
        print(f"  資料範圍: {data.index[0].strftime('%Y-%m-%d')} ~ {data.index[-1].strftime('%Y-%m-%d')}")

        # 2. 計算技術指標
        print("\n[2/6] 計算技術指標...")
        df = self.calculate_indicators(data)
        close_col = 'Close' if 'Close' in df.columns else 'close'
        print(f"  MA: {df['MA'].iloc[-1]:.2f}")
        print(f"  RSI: {df['RSI'].iloc[-1]:.2f}")
        print(f"  MACD: {df['MACD'].iloc[-1]:.4f}")

        # 3. 趨勢預測 (技術指標)
        print("\n[3/6] 分析股價趨勢 (技術指標)...")
        prediction = self.predict_trend(df)
        print(f"  預測方向: {prediction['prediction']}")
        print(f"  信心度: {prediction['confidence']:.1f}%")

        # 4. 技術訊號
        print("\n[4/6] 生成本週交易訊號...")
        current_signal = self.generate_signal(df, len(df) - 1)
        signal_map = {
            0: "觀望 (無動作)",
            1: "買入訊號",
            -1: "賣出訊號",
            2: "空單出場",
            -2: "多單出場"
        }
        print(f"  本週訊號: {signal_map.get(current_signal, '未知')}")
        print(f"  停利模式: {'動態停利 (趨勢反轉)' if self.use_dynamic_tp else '固定停利 (' + str(self.take_profit_pct*100) + '%)'}")

        # 5. 回測驗證
        print("\n[5/6] 執行回測驗證...")
        backtest_result = self.backtest(data)
        print(f"  歷史勝率: {backtest_result['win_rate']:.1f}%")
        print(f"  獲利因子: {backtest_result['profit_factor']:.2f}")

        # 6. ARIMAX 分析 (可選)
        arimax_results = None
        if include_arimax:
            print("\n[6/6] 執行 ARIMAX 多因子分析...")
            try:
                self.arimax_strategy = ARIMAXStrategy(target_stock=target_stock)
                arimax_info, arimax_predictions = self.arimax_strategy.run_simple_model_strategy()
                self.arimax_predictions = arimax_predictions
                arimax_results = {
                    'model_info': arimax_info,
                    'predictions': arimax_predictions
                }
            except Exception as e:
                print(f"  ARIMAX 分析失敗: {e}")
        else:
            print("\n[6/6] 跳過 ARIMAX 分析")

        # 組合結果
        model_info = {
            'strategy_name': '多因子股價預測策略',
            'target': '股價預測',
            'target_stock': target_stock,
            'parameters': {
                'ma_period': self.ma_period,
                'rsi_period': self.rsi_period,
                'macd_params': f"{self.macd_fast}/{self.macd_slow}/{self.macd_signal}",
                'stop_loss': f"{self.stop_loss_pct * 100}%",
                'take_profit': f"{self.take_profit_pct * 100}%",
                'dynamic_tp': self.use_dynamic_tp
            }
        }

        results = {
            'prediction': prediction,
            'current_signal': current_signal,
            'signal_description': signal_map.get(current_signal, '未知'),
            'backtest': backtest_result,
            'latest_indicators': {
                'close': df[close_col].iloc[-1],
                'MA': df['MA'].iloc[-1],
                'RSI': df['RSI'].iloc[-1],
                'MACD': df['MACD'].iloc[-1],
                'MACD_Signal': df['MACD_Signal'].iloc[-1],
                'BB_Upper': df['BB_Upper'].iloc[-1],
                'BB_Lower': df['BB_Lower'].iloc[-1]
            },
            'arimax': arimax_results,
            'recommendation': self._generate_recommendation(prediction, current_signal, arimax_results)
        }

        print("\n" + "=" * 70)
        print(f"★ 本週建議: {results['recommendation']}")
        print("=" * 70)

        return model_info, results

    def _generate_recommendation(self, prediction, signal, arimax_results=None):
        """
        生成交易建議 (整合技術分析 + ARIMAX)

        Args:
            prediction: 技術指標趨勢預測結果
            signal: 當前技術訊號
            arimax_results: ARIMAX 分析結果

        Returns:
            建議文字
        """
        trend = prediction['prediction']
        confidence = prediction['confidence']

        # 基礎建議 (技術分析)
        if signal == 1:
            base_rec = f"技術面建議買入 - 趨勢{trend}，信心度{confidence:.1f}%"
        elif signal == -1:
            base_rec = f"技術面建議賣出 - 趨勢{trend}，信心度{confidence:.1f}%"
        elif signal == 2 or signal == -2:
            base_rec = f"技術面建議平倉 - 趨勢轉變"
        else:
            if trend == 'BULLISH' and confidence > 50:
                base_rec = f"觀望偏多 - 等待買入訊號確認"
            elif trend == 'BEARISH' and confidence > 50:
                base_rec = f"觀望偏空 - 等待賣出訊號確認"
            else:
                base_rec = f"維持觀望 - 趨勢不明確"

        # 加入 ARIMAX 預測
        if arimax_results and 'predictions' in arimax_results:
            preds = arimax_results['predictions']

            ai_signals = []

            if 'linear_model' in preds:
                lr_pred = preds['linear_model']['prediction']
                lr_name = preds['linear_model']['name']
                if lr_pred > 0.01:
                    ai_signals.append(f"{lr_name}看漲({lr_pred*100:.2f}%)")
                elif lr_pred < -0.01:
                    ai_signals.append(f"{lr_name}看跌({lr_pred*100:.2f}%)")
                else:
                    ai_signals.append(f"{lr_name}中性")

            if 'arimax' in preds:
                arimax_pred = preds['arimax']['prediction']
                if arimax_pred > 0.01:
                    ai_signals.append(f"ARIMAX看漲({arimax_pred*100:.2f}%)")
                elif arimax_pred < -0.01:
                    ai_signals.append(f"ARIMAX看跌({arimax_pred*100:.2f}%)")
                else:
                    ai_signals.append(f"ARIMAX中性")

            if ai_signals:
                base_rec += f" | AI模型: {', '.join(ai_signals)}"

        return base_rec


# =============================================================================
# 主程式
# =============================================================================

def main():
    """
    主程式 - 策略執行範例
    """
    print("=" * 70)
    print("多因子股價預測策略 - 完整示範")
    print("=" * 70)

    # 範例 1: 固定停利策略
    print("\n--- 範例 1: 固定停利 (6.0%) ---")
    strategy_fixed = StockPredictionStrategy(use_dynamic_tp=False)
    print(f"  停利模式: 固定停利 ({strategy_fixed.take_profit_pct*100}%)")

    # 範例 2: 動態停利策略
    print("\n--- 範例 2: 動態停利 (MA 趨勢停利) ---")
    strategy_dynamic = StockPredictionStrategy(use_dynamic_tp=True)
    print(f"  停利模式: 動態停利 (趨勢反轉)")

    # 執行完整週策略
    print("\n" + "=" * 70)
    print("執行完整週策略分析")
    print("=" * 70)

    model_info, results = strategy_dynamic.run_weekly_strategy(
        data=None,
        target_stock='mu',
        include_arimax=True
    )

    return strategy_dynamic, model_info, results


if __name__ == "__main__":
    strategy, model_info, results = main()


In [15]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import warnings
import yfinance as yf
import pandas_datareader.data as web

# 模型相關庫
from statsmodels.tsa.arima.model import ARIMA
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error

warnings.filterwarnings('ignore')

# =============================================================================
# 1. 技術指標計算函數
# =============================================================================
def calculate_MA(prices, period=10):
    prices = np.array(prices, dtype=float)
    ma = np.full(len(prices), np.nan)
    for i in range(period - 1, len(prices)):
        ma[i] = np.mean(prices[i - period + 1:i + 1])
    return ma

def calculate_EMA(prices, period=12):
    prices = np.array(prices, dtype=float)
    ema = np.full(len(prices), np.nan)
    multiplier = 2 / (period + 1)
    ema[period - 1] = np.mean(prices[:period])
    for i in range(period, len(prices)):
        ema[i] = (prices[i] - ema[i - 1]) * multiplier + ema[i - 1]
    return ema

def calculate_RSI(prices, period=14):
    prices = np.array(prices, dtype=float)
    rsi = np.full(len(prices), np.nan)
    deltas = np.diff(prices)
    gains = np.where(deltas > 0, deltas, 0)
    losses = np.where(deltas < 0, -deltas, 0)

    avg_gain = np.mean(gains[:period])
    avg_loss = np.mean(losses[:period])

    if avg_loss == 0:
        rsi[period] = 100
    else:
        rs = avg_gain / avg_loss
        rsi[period] = 100 - (100 / (1 + rs))

    for i in range(period + 1, len(prices)):
        gain = gains[i-1]
        loss = losses[i-1]
        avg_gain = (avg_gain * (period - 1) + gain) / period
        avg_loss = (avg_loss * (period - 1) + loss) / period

        if avg_loss == 0:
            rsi[i] = 100
        else:
            rs = avg_gain / avg_loss
            rsi[i] = 100 - (100 / (1 + rs))
    return rsi

def calculate_MACD(prices, fast_period=12, slow_period=26, signal_period=9):
    prices = np.array(prices, dtype=float)
    ema_fast = calculate_EMA(prices, fast_period)
    ema_slow = calculate_EMA(prices, slow_period)
    macd = ema_fast - ema_slow

    signal = np.full(len(prices), np.nan)
    valid_start = slow_period - 1
    if len(macd) > valid_start + signal_period:
        valid_macd = macd[valid_start:]
        signal_values = calculate_EMA(valid_macd, signal_period)
        signal[valid_start:] = signal_values

    return macd, signal, macd - signal

# =============================================================================
# 2. 整合策略類 (含最新邏輯 D)
# =============================================================================
class IntegratedTradingStrategy:
    def __init__(self, target_stock='AVGO', data_period_years=5):
        self.target_stock = target_stock.upper()
        self.correlated_stocks = [self.target_stock, 'NVDA', 'AMD']
        self.currency_pairs = ['DEXJPUS', 'DEXUSUK']
        self.indices = ['SP500', 'DJIA', 'VIXCLS']

        self.return_period = 5
        self.data_period_years = data_period_years

        self.ma_period = 20
        self.arimax_model = None
        self.linear_model = None
        self.best_model_name = None

    def load_data(self, start_date, end_date):
        print(f"[{self.target_stock}] 正在加載多因子數據 ({start_date} ~ {end_date})...")
        stk_data = yf.download(self.correlated_stocks, start=start_date, end=end_date, auto_adjust=False, progress=False)

        if isinstance(stk_data.columns, pd.MultiIndex):
            try:
                price_data = stk_data['Adj Close'].copy()
            except KeyError:
                print("警告: 找不到 'Adj Close'，嘗試使用 'Close'")
                price_data = stk_data['Close'].copy()
        else:
            price_data = pd.DataFrame(stk_data['Adj Close'])
            price_data.columns = [self.target_stock]

        price_data = price_data.ffill().bfill()

        try:
            ccy_data = web.DataReader(self.currency_pairs, 'fred', start_date, end_date).ffill()
            idx_data = web.DataReader(self.indices, 'fred', start_date, end_date).ffill()
        except Exception as e:
            print(f"FRED 數據下載失敗: {e} (使用隨機數據模擬)")
            dates = price_data.index
            ccy_data = pd.DataFrame(np.random.randn(len(dates), len(self.currency_pairs)), index=dates, columns=self.currency_pairs)
            idx_data = pd.DataFrame(np.random.randn(len(dates), len(self.indices)), index=dates, columns=self.indices)

        return price_data, ccy_data, idx_data

    def prepare_features(self, stk_prices, ccy_data, idx_data):
        print("正在準備 ARIMAX/LR 特徵工程...")
        all_data = pd.concat([stk_prices, ccy_data, idx_data], axis=1).ffill().dropna()

        target_series = all_data[self.target_stock]
        Y = np.log(target_series).diff(self.return_period).shift(-self.return_period)
        Y.name = 'Target_Return'

        X1 = np.log(all_data[self.correlated_stocks]).diff(self.return_period)
        X2 = np.log(all_data[self.currency_pairs]).diff(self.return_period)
        X3 = np.log(all_data[self.indices]).diff(self.return_period)

        periods = [self.return_period, self.return_period*3, self.return_period*6]
        X4_list = []
        for p in periods:
            col = np.log(target_series).diff(p)
            col.name = f'Momentum_{p}d'
            X4_list.append(col)
        X4 = pd.concat(X4_list, axis=1)

        X = pd.concat([X1, X2, X3, X4], axis=1)
        dataset = pd.concat([Y, X], axis=1).dropna()
        dataset = dataset.iloc[::self.return_period, :]

        return dataset.drop(columns=['Target_Return']), dataset['Target_Return'], all_data

    def train_models(self, X_train, Y_train, X_test, Y_test):
        print("-> 正在訓練 AI 模型 (LR, LASSO, ARIMAX)...")

        models = {
            'LR': LinearRegression(),
            'LASSO': Lasso(alpha=0.0001, max_iter=10000),
            'EN': ElasticNet(alpha=0.0001, max_iter=10000)
        }

        best_mse = float('inf')
        for name, model in models.items():
            model.fit(X_train, Y_train)
            pred = model.predict(X_test)
            mse = mean_squared_error(Y_test, pred)

            if mse < best_mse:
                best_mse = mse
                self.best_model_name = name
                self.linear_model = model

        print(f"-> 最佳線性模型: {self.best_model_name} (MSE: {best_mse:.6f})")

        corrs = X_train.corrwith(Y_train).abs().sort_values(ascending=False)
        top_features = corrs.head(3).index.tolist()
        self.arimax_exog_vars = top_features

        try:
            model_arima = ARIMA(endog=Y_train, exog=X_train[top_features], order=(1,0,1))
            self.arimax_model = model_arima.fit()
            arima_pred = self.arimax_model.forecast(steps=len(X_test), exog=X_test[top_features])
            arima_mse = mean_squared_error(Y_test, arima_pred)
            print(f"-> ARIMAX (1,0,1) MSE: {arima_mse:.6f}")
        except Exception as e:
            print(f"ARIMAX 訓練失敗: {e}")

    def analyze_technical_status(self, full_price_data):
        prices = full_price_data[self.target_stock].values

        ma = calculate_MA(prices, self.ma_period)
        rsi = calculate_RSI(prices, 14)
        macd, signal, hist = calculate_MACD(prices)

        curr_price = prices[-1]
        curr_ma = ma[-1]
        curr_rsi = rsi[-1]
        curr_hist = hist[-1]

        status = {
            'price': curr_price,
            'rsi': curr_rsi,
            'macd_hist': curr_hist,
            'ma_trend': 'BULL' if curr_price > curr_ma else 'BEAR'
        }

        score = 0
        if curr_price > curr_ma: score += 1
        else: score -= 1

        if curr_hist > 0: score += 1
        else: score -= 1

        if 40 < curr_rsi < 60: score += 0
        elif curr_rsi >= 60: score += 1
        elif curr_rsi <= 40: score -= 1

        status['tech_score'] = score
        return status

    def run_strategy(self):
        print("\n" + "="*60)
        print(f"執行每週策略分析: {self.target_stock}")
        print("="*60)

        end_date = datetime.now()
        start_date = end_date - timedelta(days=365 * self.data_period_years)

        stk_prices, ccy_data, idx_data = self.load_data(start_date, end_date)
        X, Y, raw_data = self.prepare_features(stk_prices, ccy_data, idx_data)

        train_size = int(len(X) * 0.85)
        X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]
        Y_train, Y_test = Y.iloc[:train_size], Y.iloc[train_size:]

        self.train_models(X_train, Y_train, X_test, Y_test)

        current_features = X.iloc[-1:].values.reshape(1, -1)
        current_features_df = X.iloc[-1:]

        pred_lr = self.linear_model.predict(current_features)[0]

        pred_arima = 0
        if self.arimax_model:
            exog_latest = current_features_df[self.arimax_exog_vars]
            pred_arima = self.arimax_model.forecast(steps=1, exog=exog_latest).iloc[0]

        avg_pred_return = (pred_lr + pred_arima) / 2
        print(f"\n[AI 預測結果]")
        print(f"線性模型預測: {pred_lr*100:.2f}%")
        print(f"ARIMAX 預測 : {pred_arima*100:.2f}%")
        print(f"-> 綜合預測下週漲跌: {avg_pred_return*100:.2f}%")

        tech_status = self.analyze_technical_status(raw_data)
        print(f"\n[技術面狀態]")
        print(f"當前價格: {tech_status['price']:.2f}")
        print(f"RSI: {tech_status['rsi']:.2f}")
        print(f"MACD柱狀圖: {tech_status['macd_hist']:.4f}")
        print(f"技術評分: {tech_status['tech_score']} (正為多，負為空)")

        # ==========================================
        # 最終決策 (AI Confidence Filter Logic)
        # ==========================================
        print("\n" + "-"*40)
        print("最終交易建議")
        print("-"*40)

        decision = "觀望 (Wait)"
        action = "無動作"

        # 邏輯 A: 技術面與 AI 共振 (最佳進場點)
        if tech_status['tech_score'] >= 1 and avg_pred_return > 0.005:
            decision = "強烈買入 (Strong Buy)"
            action = "進場做多 (趨勢確認)"

        # 邏輯 B: AI 強力看漲，技術面尚未跟上 (左側交易/潛伏)
        elif avg_pred_return > 0.015 and tech_status['rsi'] > 30:
            decision = "潛伏買入 (Accumulate)"
            action = "小部位建倉 (博反彈)"

        # 邏輯 C: 技術面轉弱，AI 也看空
        elif tech_status['tech_score'] <= -1 and avg_pred_return < -0.005:
            decision = "賣出/做空 (Sell)"
            action = "清倉或反手"

        # 邏輯 D: 技術面過熱，AI 預警回調 (獲利了結訊號)  <-- 新增部分
        elif tech_status['tech_score'] >= 2 and avg_pred_return < 0:
            decision = "預警回調 (Caution)"
            action = "多單減碼 / 移動停利 / 禁止追高"

        print(f"決策: {decision}")
        print(f"行動: {action}")
        print("="*60)

# =============================================================================
# 執行區
# =============================================================================
if __name__ == "__main__":
    # 您可以修改這裡的股票代碼
    bot = IntegratedTradingStrategy(target_stock='AVGO', data_period_years=2)
    bot.run_strategy()


執行每週策略分析: AVGO
[AVGO] 正在加載多因子數據 (2023-11-29 08:15:50.444338 ~ 2025-11-28 08:15:50.444338)...
正在準備 ARIMAX/LR 特徵工程...
-> 正在訓練 AI 模型 (LR, LASSO, ARIMAX)...
-> 最佳線性模型: LR (MSE: 0.003945)
-> ARIMAX (1,0,1) MSE: 0.003424

[AI 預測結果]
線性模型預測: -1.57%
ARIMAX 預測 : 0.59%
-> 綜合預測下週漲跌: -0.49%

[技術面狀態]
當前價格: 397.57
RSI: 66.89
MACD柱狀圖: 4.1179
技術評分: 3 (正為多，負為空)

----------------------------------------
最終交易建議
----------------------------------------
決策: 預警回調 (Caution)
行動: 多單減碼 / 移動停利 / 禁止追高


In [12]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import warnings
import yfinance as yf
import pandas_datareader.data as web

# 模型相关库 deepseek
from statsmodels.tsa.arima.model import ARIMA
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, cross_val_score

warnings.filterwarnings('ignore')

# =============================================================================
# 技术指标计算函数
# =============================================================================
def calculate_MA(prices, period=10):
    """计算移动平均线"""
    prices = np.array(prices, dtype=float)
    ma = np.full(len(prices), np.nan)
    for i in range(period - 1, len(prices)):
        ma[i] = np.mean(prices[i - period + 1:i + 1])
    return ma

def calculate_EMA(prices, period=12):
    """计算指数移动平均线"""
    prices = np.array(prices, dtype=float)
    ema = np.full(len(prices), np.nan)
    multiplier = 2 / (period + 1)
    ema[period - 1] = np.mean(prices[:period])
    for i in range(period, len(prices)):
        ema[i] = (prices[i] - ema[i - 1]) * multiplier + ema[i - 1]
    return ema

def calculate_RSI(prices, period=14):
    """计算相对强弱指数"""
    prices = np.array(prices, dtype=float)
    rsi = np.full(len(prices), np.nan)
    deltas = np.diff(prices)
    gains = np.where(deltas > 0, deltas, 0)
    losses = np.where(deltas < 0, -deltas, 0)
    for i in range(period, len(prices)):
        avg_gain = np.mean(gains[i - period:i])
        avg_loss = np.mean(losses[i - period:i])
        if avg_loss == 0:
            rsi[i] = 100
        else:
            rs = avg_gain / avg_loss
            rsi[i] = 100 - (100 / (1 + rs))
    return rsi

def calculate_MACD(prices, fast_period=12, slow_period=26, signal_period=9):
    """计算MACD指标"""
    prices = np.array(prices, dtype=float)
    ema_fast = calculate_EMA(prices, fast_period)
    ema_slow = calculate_EMA(prices, slow_period)
    macd = ema_fast - ema_slow
    signal = np.full(len(prices), np.nan)
    valid_macd = macd[~np.isnan(macd)]
    if len(valid_macd) >= signal_period:
        signal_values = calculate_EMA(valid_macd, signal_period)
        start_idx = len(prices) - len(valid_macd)
        signal[start_idx:] = signal_values
    return macd, signal, macd - signal

def calculate_Bollinger_Bands(prices, period=20, num_std=2):
    """计算布林带"""
    prices = np.array(prices, dtype=float)
    middle = calculate_MA(prices, period)
    upper = np.full(len(prices), np.nan)
    lower = np.full(len(prices), np.nan)
    for i in range(period - 1, len(prices)):
        std = np.std(prices[i - period + 1:i + 1])
        upper[i] = middle[i] + num_std * std
        lower[i] = middle[i] - num_std * std
    return upper, middle, lower

# =============================================================================
# 整合策略类
# =============================================================================
class IntegratedTradingStrategy:
    """
    整合多因子ARIMAX模型和技术指标的交易策略
    """

    def __init__(self, target_stock='avgo', data_period_years=5):
        # 多因子ARIMAX参数
        self.target_stock = target_stock.upper()
        self.correlated_stocks = ['mu', 'GOOGL']
        self.currency_pairs = ['DEXJPUS', 'DEXUSUK']
        self.indices = ['SP500', 'DJIA', 'VIXCLS']
        self.return_period = 5
        self.data_period_years = data_period_years

        # 技术指标参数
        self.ma_period = 10
        self.rsi_period = 14
        self.macd_fast = 12
        self.macd_slow = 26
        self.macd_signal = 9
        self.stop_loss_pct = 0.03
        self.take_profit_pct = 0.06
        self.use_dynamic_tp = True

        # 策略状态
        self.position = 0
        self.entry_price = 0
        self.trades = []
        self.total_profit = 0

        # 模型
        self.arimax_model = None
        self.linear_model = None
        self.best_model_name = None

    def load_data(self, start_date, end_date):
        """加载股票、货币和指数数据"""
        print("正在加载多因子数据...")

        stk_tickers = [self.target_stock] + self.correlated_stocks
        stk_data_multi_index = yf.download(stk_tickers, start=start_date, end=end_date, progress=False)

        ccy_data = web.DataReader(self.currency_pairs, 'fred', start_date, end_date)
        idx_data = web.DataReader(self.indices, 'fred', start_date, end_date)

        return stk_data_multi_index, ccy_data, idx_data

    def prepare_features(self, stk_data_multi_index, ccy_data, idx_data):
        """准备特征变量和目标变量"""
        print("正在准备ARIMAX/LR特征...")

        # 检查数据可用性
        if 'Adj Close' in stk_data_multi_index.columns.get_level_values(0):
            stk_price_data = stk_data_multi_index['Adj Close'].copy()
        elif 'Close' in stk_data_multi_index.columns.get_level_values(0):
            print("警告: 'Adj Close' 数据可能不完整，使用 'Close' 价格")
            stk_price_data = stk_data_multi_index['Close'].copy()
        else:
            raise ValueError("错误: 找不到价格数据")

        # 合并所有数据
        all_data = pd.concat([stk_price_data, ccy_data, idx_data], axis=1).ffill().dropna()

        # 目标变量
        Y = np.log(all_data.loc[:, self.target_stock]).diff(self.return_period).shift(-self.return_period)
        Y.name = f'{self.target_stock}_pred'

        # 特征工程
        X1 = np.log(all_data.loc[:, self.correlated_stocks]).diff(self.return_period)
        X2 = np.log(all_data.loc[:, self.currency_pairs]).diff(self.return_period)
        X3 = np.log(all_data.loc[:, self.indices]).diff(self.return_period)

        X4 = pd.concat([
            np.log(all_data.loc[:, self.target_stock]).diff(i)
            for i in [self.return_period, self.return_period*3,
                     self.return_period*6, self.return_period*12]
        ], axis=1)

        X4.columns = [f'{self.target_stock}_DT', f'{self.target_stock}_3DT',
                      f'{self.target_stock}_6DT', f'{self.target_stock}_12DT']

        X = pd.concat([X1, X2, X3, X4], axis=1)

        # 创建最终数据集
        dataset = pd.concat([Y, X], axis=1).dropna().iloc[::self.return_period, :]
        Y_final = dataset.loc[:, Y.name]
        X_final = dataset.loc[:, X.columns]

        return X_final, Y_final, dataset

    def train_linear_models(self, X_train, Y_train, X_test, Y_test):
        """训练并比较线性模型"""
        print("-> 正在训练LR, LASSO, EN模型...")

        models = [
            ('LR', LinearRegression()),
            ('LASSO', Lasso(random_state=42, max_iter=10000)),
            ('EN', ElasticNet(random_state=42, max_iter=10000))
        ]

        best_test_mse = float("inf")
        best_model_name = None
        best_regressor = None

        for name, model in models:
            try:
                model.fit(X_train, Y_train)
                test_pred = model.predict(X_test)
                test_mse = mean_squared_error(Y_test, test_pred)

                print(f'   - {name}: Test MSE={test_mse:.6f}')

                if test_mse < best_test_mse:
                    best_test_mse = test_mse
                    best_model_name = name
                    best_regressor = model
            except Exception as e:
                print(f'   - {name}: 训练失败 ({e})')
                continue

        if best_regressor:
            print(f"-> 最佳线性模型: {best_model_name} (Test MSE: {best_test_mse:.6f})")

            # 显示回归方程
            if hasattr(best_regressor, 'coef_') and hasattr(best_regressor, 'intercept_'):
                coefficients = best_regressor.coef_
                intercept = best_regressor.intercept_
                feature_names = X_train.columns

                print("\n-> 最佳模型的回归方程:")
                equation = f"   Y_pred = {intercept:.6f}"
                for name, coef in zip(feature_names, coefficients):
                    if abs(coef) > 1e-6:
                        sign = "+" if coef > 0 else "-"
                        equation += f" {sign} {abs(coef):.6f} * {name}"
                print(equation)

        return best_regressor, best_model_name, best_test_mse

    def train_arimax_model(self, X_train, Y_train, order=(1, 0, 0)):
        """训练ARIMAX模型"""
        print("-> 正在训练ARIMAX模型...")

        exogenous_vars = [col for col in X_train.columns if not col.startswith(self.target_stock)]
        X_train_arima = X_train.loc[:, exogenous_vars]

        try:
            model = ARIMA(endog=Y_train, exog=X_train_arima, order=order)
            model_fit = model.fit()
            return model_fit, exogenous_vars
        except Exception as e:
            print(f"   - ARIMAX模型训练失败: {e}")
            return None, exogenous_vars

    def calculate_technical_indicators(self, data):
        """计算所有技术指标"""
        df = data.copy()

        close_prices = df['Close']
        df['MA'] = calculate_MA(close_prices.values, self.ma_period)
        df['RSI'] = calculate_RSI(close_prices.values, self.rsi_period)

        macd, signal, hist = calculate_MACD(close_prices.values, self.macd_fast, self.macd_slow, self.macd_signal)
        df['MACD'] = macd
        df['MACD_Signal'] = signal

        upper, middle, lower = calculate_Bollinger_Bands(close_prices.values, 20, 2)
        df['BB_Upper'] = upper
        df['BB_Middle'] = middle
        df['BB_Lower'] = lower

        return df

    def generate_technical_signal(self, df, i):
        """生成技术指标交易信号"""
        if i < 1:
            return 0

        current_price = df['Close'].iloc[i]
        prev_price = df['Close'].iloc[i-1]
        current_ma = df['MA'].iloc[i]
        prev_ma = df['MA'].iloc[i-1]

        if np.isnan(current_ma):
            return 0

        # 进场逻辑
        if self.position == 0:
            # 简单的MA交叉策略
            if prev_price <= prev_ma and current_price > current_ma:
                return 1  # 买入
            if prev_price >= prev_ma and current_price < current_ma:
                return -1  # 卖出
            return 0

        # 出场逻辑
        # 止损检查
        if self.position == 1 and current_price <= self.entry_price * (1 - self.stop_loss_pct):
            return -2  # 多单止损
        if self.position == -1 and current_price >= self.entry_price * (1 + self.stop_loss_pct):
            return 2  # 空单止损

        # 停利检查
        if self.use_dynamic_tp:
            # 动态停利：趋势反转时出场
            if self.position == 1 and prev_price >= prev_ma and current_price < current_ma:
                return -2
            if self.position == -1 and prev_price <= prev_ma and current_price > current_ma:
                return 2
        else:
            # 固定停利
            if self.position == 1 and current_price >= self.entry_price * (1 + self.take_profit_pct):
                return -2
            if self.position == -1 and current_price <= self.entry_price * (1 - self.take_profit_pct):
                return 2

        return 0

    def execute_trade(self, signal, price, time):
        """执行交易"""
        if signal == 1:
            self.position = 1
            self.entry_price = price
            self.trades.append({'action': 'BUY', 'price': price, 'time': time})
        elif signal == -1:
            self.position = -1
            self.entry_price = price
            self.trades.append({'action': 'SELL', 'price': price, 'time': time})
        elif signal == -2:
            profit = price - self.entry_price
            self.total_profit += profit
            self.trades.append({'action': 'EXIT_LONG', 'profit': profit, 'time': time})
            self.position = 0
            self.entry_price = 0
        elif signal == 2:
            profit = self.entry_price - price
            self.total_profit += profit
            self.trades.append({'action': 'EXIT_SHORT', 'profit': profit, 'time': time})
            self.position = 0
            self.entry_price = 0

    def get_performance_report(self):
        """生成绩效报告"""
        if not self.trades:
            return {
                'total_trades': 0,
                'total_profit': 0,
                'win_rate': 0,
                'profit_factor': 0
            }

        # 只计算有利润的交易
        exit_trades = [t for t in self.trades if 'profit' in t]
        if not exit_trades:
            return {
                'total_trades': 0,
                'total_profit': 0,
                'win_rate': 0,
                'profit_factor': 0
            }

        profits = [t.get('profit', 0) for t in exit_trades]
        wins = [p for p in profits if p > 0]
        losses = [p for p in profits if p <= 0]
        total_loss = abs(sum(losses))

        return {
            'total_trades': len(exit_trades),
            'winning_trades': len(wins),
            'losing_trades': len(losses),
            'total_profit': self.total_profit,
            'win_rate': len(wins) / len(exit_trades) * 100 if exit_trades else 0,
            'profit_factor': sum(wins) / total_loss if total_loss != 0 else float('inf')
        }

    def run_complete_strategy(self):
        """执行完整策略"""
        print("\n" + "=" * 60)
        print(f"整合交易策略执行 - {self.target_stock}")
        print("=" * 60)

        # 1. 设置日期范围
        end_date = datetime.now().strftime('%Y-%m-%d')
        start_date = (datetime.now() - timedelta(days=365 * self.data_period_years)).strftime('%Y-%m-%d')

        # 2. 加载数据
        stk_data, ccy_data, idx_data = self.load_data(start_date, end_date)

        # 3. 准备多因子特征
        X, Y, dataset = self.prepare_features(stk_data, ccy_data, idx_data)

        # 4. 划分训练测试集
        train_size = int(len(X) * 0.8)
        X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]
        Y_train, Y_test = Y.iloc[:train_size], Y.iloc[train_size:]

        print(f"-> 训练集大小: {len(X_train)} 周")
        print(f"-> 测试集大小: {len(X_test)} 周")

        # 5. 训练线性模型
        self.linear_model, self.best_model_name, linear_mse = self.train_linear_models(
            X_train, Y_train, X_test, Y_test)

        # 6. 训练ARIMAX模型
        self.arimax_model, arimax_exog_vars = self.train_arimax_model(X_train, Y_train, order=(2, 0, 1))

        # 7. 模型评估
        if self.arimax_model:
            X_test_arima = X_test.loc[:, arimax_exog_vars]
            test_forecast_arimax = self.arimax_model.forecast(steps=len(X_test), exog=X_test_arima)
            arimax_test_mse = mean_squared_error(Y_test, test_forecast_arimax)
            print(f"-> ARIMAX (2,0,1) Test MSE: {arimax_test_mse:.6f}")

        # 8. 生成预测信号
        current_features = X.iloc[-1:].fillna(0)

        prediction_signals = {}

        if self.linear_model:
            lr_prediction = self.linear_model.predict(current_features)[0]
            prediction_signals['Linear_Model'] = lr_prediction
            print(f"\n-> 线性模型 ({self.best_model_name}) 下周预测回报率: {lr_prediction:.4f}")

        if self.arimax_model:
            arimax_prediction = self.arimax_model.forecast(
                steps=1, exog=current_features.loc[:, arimax_exog_vars])[0]
            prediction_signals['ARIMAX_Model'] = arimax_prediction
            print(f"-> ARIMAX 下周预测回报率: {arimax_prediction:.4f}")

        # 9. 技术指标分析
        print("\n" + "-" * 40)
        print("技术指标分析")
        print("-" * 40)

        # 获取目标股票数据计算技术指标
        stock_data = yf.download(self.target_stock, start=start_date, end=end_date, progress=False)
        tech_data = self.calculate_technical_indicators(stock_data)

        # 生成当前技术信号
        current_tech_signal = self.generate_technical_signal(tech_data, len(tech_data)-1)
        signal_descriptions = {
            1: "买入信号",
            -1: "卖出信号",
            -2: "多单出场",
            2: "空单出场",
            0: "无信号"
        }
        print(f"-> 技术指标信号: {signal_descriptions.get(current_tech_signal, '未知')}")

        # 10. 综合决策
        print("\n" + "=" * 40)
        print("综合交易决策")
        print("=" * 40)

        # 基于预测回报率生成综合信号
        avg_prediction = np.mean(list(prediction_signals.values())) if prediction_signals else 0

        final_signal = 0
        if avg_prediction > 0.01 and current_tech_signal == 1:
            final_signal = 1
            decision = "强烈买入"
        elif avg_prediction > 0.005:
            final_signal = 1
            decision = "温和买入"
        elif avg_prediction < -0.01 and current_tech_signal == -1:
            final_signal = -1
            decision = "强烈卖出"
        elif avg_prediction < -0.005:
            final_signal = -1
            decision = "温和卖出"
        else:
            decision = "保持观望"

        current_price = stock_data['Close'].iloc[-1]
        print(f"当前价格: ${current_price:.2f}")
        print(f"平均预测回报: {avg_prediction:.4f}")
        print(f"最终决策: {decision}")

        # 11. 如果有信号，执行交易
        if final_signal != 0:
            current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            self.execute_trade(final_signal, current_price, current_time)

        # 12. 绩效报告
        performance = self.get_performance_report()
        print(f"\n策略绩效:")
        print(f"总交易次数: {performance['total_trades']}")
        print(f"盈利交易: {performance['winning_trades']}次")
        print(f"亏损交易: {performance['losing_trades']}次")
        print(f"胜率: {performance['win_rate']:.1f}%")
        print(f"总利润: ${performance['total_profit']:.2f}")
        print(f"盈利因子: {performance['profit_factor']:.2f}")

        print("\n" + "=" * 60)
        print("策略执行完成!")
        print("=" * 60)

        return {
            'predictions': prediction_signals,
            'technical_signal': current_tech_signal,
            'final_decision': decision,
            'performance': performance
        }

# 使用示例
if __name__ == "__main__":
    # 创建整合策略实例
    strategy = IntegratedTradingStrategy(target_stock='avgo', data_period_years=3)

    # 执行完整策略
    try:
        results = strategy.run_complete_strategy()

        print("\n建议执行计划:")
        print("1. 每周三执行此策略获取交易信号")
        print("2. 根据综合决策执行交易")
        print("3. 设置止损: 3%, 止盈: 6% (动态)")
        print("4. 定期重新训练模型 (建议每月)")

    except Exception as e:
        print(f"策略执行出错: {e}")
        print("请检查网络连接和数据源可用性")


整合交易策略执行 - AVGO
正在加载多因子数据...
正在准备ARIMAX/LR特征...
警告: 'Adj Close' 数据可能不完整，使用 'Close' 价格
策略执行出错: "['avgo'] not in index"
请检查网络连接和数据源可用性
